In [1]:
import tensorflow as tf
from tfx import v1 as tfx

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

print('TFX version: {}'.format(tfx.__version__))
print('TensorFlow version: {}'.format(tf.__version__))

TFX version: 1.3.3
TensorFlow version: 2.6.2


In [2]:
import os
from absl import logging

PIPELINE_NAME = "iris"
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

DATA_ROOT = os.path.join("data")

logging.set_verbosity(logging.INFO)

# Retrieve data

In [3]:
import csv
import urllib.request

_data_url = 'https://forge.scilab.org/index.php/p/rdataset/source/file/master/csv/datasets/iris.csv'
_download_filepath = os.path.join(DATA_ROOT, "iris.csv")
_data_filepath = os.path.join(DATA_ROOT, "data.csv")

urllib.request.urlretrieve(_data_url, _download_filepath)

with open(_download_filepath, 'r') as infile, open(_data_filepath, 'w') as outfile:
    r = csv.reader(infile)
    w = csv.writer(outfile)

    next(r, None)
    w.writerow(["id", "Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width", "Species"])

    for row in r:
        w.writerow(row)

os.remove(_download_filepath)

In [4]:
!head {_data_filepath}

# Create TFX pipeline and run it

In [5]:
_module_file = 'iris_utils.py'

In [6]:
import tensorflow_model_analysis as tfma

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
    """Implements the penguin pipeline with TFX."""
    # Brings data into the pipeline or otherwise joins/converts training data.
    example_gen = tfx.components.CsvExampleGen(input_base=data_root)

    # Computes statistics over data for visualization and example validation.
    statistics_gen = tfx.components.StatisticsGen(
        examples=example_gen.outputs['examples'])

    # Generate a schema based on your data statistics.
    # A schema defines the expected bounds, types, and properties of the features in your dataset.
    schema_gen = tfx.components.SchemaGen(
        statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

    # Performs anomaly detection based on statistics and data schema.
    example_validator = tfx.components.ExampleValidator(
        statistics=statistics_gen.outputs['statistics'],
        schema=schema_gen.outputs['schema'])

    # Transforms input data using preprocessing_fn in the 'module_file'.
    transform = tfx.components.Transform(
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        materialize=False,
        module_file=module_file)

    # Uses user-provided Python function that trains a model.
    trainer = tfx.components.Trainer(
        module_file=module_file,
        examples=example_gen.outputs['examples'],
        schema=schema_gen.outputs['schema'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=tfx.proto.TrainArgs(num_steps=2000),
        eval_args=tfx.proto.EvalArgs(num_steps=5))

    # Pushes the model to a filesystem destination.
    pusher = tfx.components.Pusher(
        model=trainer.outputs['model'],
        push_destination=tfx.proto.PushDestination(filesystem=tfx.proto.PushDestination
                                                   .Filesystem(base_directory=serving_model_dir))
    )

    components = [
        example_gen,
        statistics_gen,
        schema_gen,
        example_validator,
        transform,
        trainer,
        pusher,
    ]

    return tfx.dsl.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        metadata_connection_config=tfx.orchestration.metadata.sqlite_metadata_connection_config(metadata_path),
        components=components)

In [ ]:
pipeline = _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH
)

tfx.orchestration.LocalDagRunner().run(pipeline)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/home/ekrem/Workplace/Python/tfx_seminar/iris_utils.py' (including modules: ['tfx_utils', 'iris_utils']).
INFO:absl:User module package has hash fingerprint version 28aa14c566d301689dd39654a55b64cc2668c092535bacda4df020cf7386af6a.
INFO:absl:Executing: ['/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/bin/python3.8', '/tmp/tmpsndvzmr4/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpgvlh12xt', '--dist-dir', '/tmp/tmp1txusg_0']
/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at 'pipelines/iris/_w

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying tfx_utils.py -> build/lib
copying iris_utils.py -> build/lib
installing to /tmp/tmpgvlh12xt
running install
running install_lib
copying build/lib/tfx_utils.py -> /tmp/tmpgvlh12xt
copying build/lib/iris_utils.py -> /tmp/tmpgvlh12xt
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpgvlh12xt/tfx_user_code_Transform-0.0+28aa14c566d301689dd39654a55b64cc2668c092535bacda4df020cf7386af6a-py3.8.egg-info
running in

/home/ekrem/.pyenv/versions/3.8.7/envs/tfx/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
INFO:absl:Successfully built user code wheel distribution at 'pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+28aa14c566d301689dd39654a55b64cc2668c092535bacda4df020cf7386af6a-py3-none-any.whl'; target user module is 'iris_utils'.
INFO:absl:Full user module path is 'iris_utils@pipelines/iris/_wheels/tfx_user_code_Trainer-0.0+28aa14c566d301689dd39654a55b64cc2668c092535bacda4df020cf7386af6a-py3-none-any.whl'
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "ExampleValidator"
  value {
    python_class_executable_spec {
      class_path: "tfx

INFO:absl:Generating examples.


running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying tfx_utils.py -> build/lib
copying iris_utils.py -> build/lib
installing to /tmp/tmpl0e7cucw
running install
running install_lib
copying build/lib/tfx_utils.py -> /tmp/tmpl0e7cucw
copying build/lib/iris_utils.py -> /tmp/tmpl0e7cucw
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpl0e7cucw/tfx_user_code_Trainer-0.0+28aa14c566d301689dd39654a55b64cc2668c092535bacda4df020cf7386af6a-py3.8.egg-info
running install_scripts
crea

...

   1/2000 [..............................] - ETA: 34:34 - loss: 1.3640 - sparse_categorical_accuracy: 0.3000

2021-11-10 14:37:45.181898: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-11-10 14:37:45.181945: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


  35/2000 [..............................] - ETA: 28s - loss: 0.8800 - sparse_categorical_accuracy: 0.6257

2021-11-10 14:37:45.500577: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-11-10 14:37:45.501513: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1748] CUPTI activity buffer flushed
2021-11-10 14:37:45.522988: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 86 callback api events and 83 activity events. 
2021-11-10 14:37:45.525715: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-11-10 14:37:45.529116: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: pipelines/iris/Trainer/model_run/11/train/plugins/profile/2021_11_10_14_37_45

2021-11-10 14:37:45.531130: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to pipelines/iris/Trainer/model_run/11/train/plugins/profile/2021_11_10_14_37_45/EkE-Legion5.trace.json.gz
2021-11-10 14:37:45.537202: I tensorflow/core/profiler/rpc/clie

1347/2000 [===================>..........] - ETA: 3s - loss: 0.0529 - sparse_categorical_accuracy: 0.9782

# Visualize pipeline

In [ ]:
from tfx.types import standard_component_specs
from tfx.orchestration.portable.mlmd import execution_lib
from tfx.orchestration.metadata import Metadata
from tfx.orchestration.experimental.interactive import visualizations
from ml_metadata.proto import metadata_store_pb2

def get_latest_artifact(component_id):
    metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(METADATA_PATH)

    with Metadata(metadata_connection_config) as metadata_handler:
        context = metadata_handler.store.get_context_by_type_and_name('node', f'{PIPELINE_NAME}.{component_id}')
        executions = metadata_handler.store.get_executions_by_context(context.id)
        latest_execution = max(executions, key=lambda e: e.last_update_time_since_epoch)

        return execution_lib.get_artifacts_dict(metadata_handler, latest_execution.id, metadata_store_pb2.Event.OUTPUT)
    
def visualize(artifact):
    visualization = visualizations.get_registry().get_visualization(artifact.type_name)
    visualization.display(artifact)
    
    
from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

## StatisticsGen

In [ ]:
statistic_artifact = get_latest_artifact("StatisticsGen")['statistics'][0]

visualize(statistic_artifact)

## SchemaGen

In [ ]:
schema_artifact = get_latest_artifact("SchemaGen")['schema'][0]

visualize(schema_artifact)

## ExampleValidator

In [ ]:
example_validator_articaft = get_latest_artifact("ExampleValidator")['anomalies'][0]

visualize(example_validator_articaft)

## Transformer

In [ ]:
# TODO

## Trainer

In [ ]:
model_run_artifact_dir = get_latest_artifact("Trainer")['model_run'][0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}